In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import zipfile
import timeit
from IPython.display import clear_output

In [2]:
ODpath = Path("../data/OD/")
OD_file = ODpath.joinpath("od_distance_1k_clean.csv.gz")
if OD_file.exists ():
    print ("OD file exist")
else:
    print ("OD file does not exist")
    

OD file exist


In [3]:
%time df = pd.read_csv(OD_file, compression='gzip', dtype={'w_geocode': str,'h_geocode':str})

Wall time: 2min 15s


#### Group the data to determine max and min values
Should be between near 1k to 70k

In [4]:
df_group = df.groupby(['w_geocode']).sum()
df_group_limited = df_group[(df_group['S000'] >= 1000)]

In [5]:
df_group_limited.S000.max()

71280

In [6]:
df_group_limited.S000.min()

1000

#### Identify all state FIPS codes in data

In [7]:
state_list = df['w_geocode'].str.slice(0,2).unique().tolist()
print (len(state_list))

48


#### Create a column to contain the state FIPS

In [8]:
df['state'] = df['w_geocode'].str.slice(0,2)

In [9]:
df.head()

,w_geocode,h_geocode,distance,w_group_count,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,w_lat,w_lon,h_lat,h_lon,state
0,010010205001001,010010201001016,5896,1076,1,1,0,0,1,0,0,0,1,0,32.457,-86.415,32.467,-86.477,01
1,010010205001001,010010201001025,6964,1076,1,1,0,0,0,1,0,0,1,0,32.457,-86.415,32.460,-86.489,01
2,010010205001001,010010201002016,7088,1076,2,0,2,0,0,0,2,0,2,0,32.457,-86.415,32.474,-86.488,01
3,010010205001001,010010201002017,6830,1076,3,3,0,0,1,2,0,0,1,2,32.457,-86.415,32.477,-86.484,01
4,010010205001001,010010201002022,7304,1076,1,0,1,0,0,1,0,0,1,0,32.457,-86.415,32.488,-86.483,01


#### Funtion to write each state file to csv

In [10]:
def write_state(df_state):
    df_state.to_csv(out_state, compression='gzip', index=None)

#### Loop through each State FIPS code and call function to write csv

In [19]:
for index in range(0,len(state_list)):
    state_id = state_list[index]
    df_state = df[df['state'] == state_id]
    outputstate = state_id + '_od_distance_1k.csv.gz'
    out_state = ODpath.joinpath(outputstate)
    write_state(df_state)